<a href="https://colab.research.google.com/github/jun-vvn/AI_lab/blob/main/%E3%82%BC%E3%83%9F5_1_%E8%BB%A2%E7%A7%BB%E5%AD%A6%E7%BF%92%E7%89%88_%E3%83%86%E3%82%B9%E3%83%88%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install japanize_matplotlib | tail -n 1
!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1
w = !apt install tree
print(w[-2])

In [ ]:
# 必要ライブラリのインポート

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

In [ ]:
# PyTorch関連ライブラリのインポート

import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets

In [ ]:
# warning表示off
import warnings
warnings.simplefilter('ignore')

# デフォルトフォントサイズ変更
plt.rcParams['font.size'] = 14

# デフォルトグラフサイズ変更
plt.rcParams['figure.figsize'] = (6,6)

# デフォルトで方眼表示ON
plt.rcParams['axes.grid'] = True

# numpyの表示桁数設定
np.set_printoptions(suppress=True, precision=5)

In [ ]:
# GPUチェック

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# 共通関数のダウンロード
!git clone https://github.com/makaishi2/pythonlibs.git

# 共通関数のロード
from pythonlibs.torch_lib1 import *

# 共通関数の存在チェック
print(README)

In [ ]:
from google.colab import files
files.upload()

In [ ]:
w = !unzip -o cifar-10.zip
print(w[-1])

In [ ]:
!tree cifar-10

In [ ]:
!tree -L 1 cifar-10

In [ ]:
import zipfile
import os
import shutil
import random
from pathlib import Path

# アップロードされたファイルのパスを修正
zip_path = '/content/cifar-10.zip'  # /mnt/data/cifar-10.zip から変更
extract_dir = '/content/cifar-10-extracted'
output_base_dir = '/content/cifar-10-dataset'

# クラス名（CIFAR-10形式）
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

# ZIPファイルを展開
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# train/test ディレクトリ構築
train_dir = os.path.join(output_base_dir, 'train')
test_dir = os.path.join(output_base_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# クラスごとにtrain/testフォルダ作成
for class_name in classes:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

# 画像をクラスごとに振り分けて移動
for class_name in classes:
    # クラスの元ディレクトリを探す
    class_dir = None
    for root, dirs, files in os.walk(extract_dir):
        if os.path.basename(root).lower() == class_name.lower():
            class_dir = root
            break
    if not class_dir:
        print(f'⚠️ クラス {class_name} のフォルダが見つかりませんでした。スキップします。')
        continue

    # 画像ファイルの取得とシャッフル
    image_files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
    random.shuffle(image_files)

    # 8:2の割合でtrain/testに分割
    split_idx = int(0.8 * len(image_files))
    train_files = image_files[:split_idx]
    test_files = image_files[split_idx:]

    # コピー実行
    for f in train_files:
        src = os.path.join(class_dir, f)
        dst = os.path.join(train_dir, class_name, f)
        shutil.copy2(src, dst)

    for f in test_files:
        src = os.path.join(class_dir, f)
        dst = os.path.join(test_dir, class_name, f)
        shutil.copy2(src, dst)

    print(f'{class_name}: Train={len(train_files)}, Test={len(test_files)}')

# 完了後、ディレクトリ構造を表示
print("\n✅ 作成されたディレクトリ構造:")
for split in ['train', 'test']:
    for class_name in classes:
        dir_path = os.path.join(output_base_dir, split, class_name)
        num_files = len(os.listdir(dir_path)) if os.path.exists(dir_path) else 0
        print(f'{split}/{class_name}: {num_files} files')

In [ ]:
!tree -L 2 cifar-10-dataset

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5)
])

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)
])

In [ ]:
# ツリーのベースディレクトリ
data_dir = '/content/cifar-10-dataset'  # data_dirを修正: '/content/cifar-10-split' から変更

# 訓練データディレクトリと検証データディレクトリの指定
import os
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')  # test_dirを修正: val -> test

# join関数の結果確認
print(train_dir, test_dir)

# 分類先クラスのリスト作成
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
train_data = datasets.ImageFolder(train_dir, transform=train_transform)
train_data2 = datasets.ImageFolder(train_dir, transform=test_transform)
test_data = datasets.ImageFolder(test_dir, transform=test_transform)

In [ ]:
print(f'訓練データ: {len(train_data)}件')
print(f'検証データ: {len(test_data)}件')

In [ ]:
# 検証データ
# 最初の10個と最後の10個の表示

plt.figure(figsize=(15, 4))
for i in range(10):
    ax = plt.subplot(2, 10, i + 1)
    image, label = test_data[i]
    img = (np.transpose(image.numpy(), (1, 2, 0)) + 1)/2
    plt.imshow(img)
    ax.set_title(classes[label])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax = plt.subplot(2, 10, i + 11)
    image, label = test_data[-i-1]
    img = (np.transpose(image.numpy(), (1, 2, 0)) + 1)/2
    plt.imshow(img)
    ax.set_title(classes[label])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

In [ ]:
# データローダー定義

batch_size = 10

# 訓練用と検証用
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(train_data2, batch_size=batch_size, shuffle=False)

# イメージ表示用
train_loader2 = DataLoader(train_data2, batch_size=50, shuffle=True)
test_loader2 = DataLoader(test_data, batch_size=50, shuffle=False)

In [ ]:
torch_seed()
show_images_labels(test_loader2, classes, None, None)

In [ ]:
# vgg19_bnをパラメータ付きで読み込む
from torchvision import models
net = models.vgg19_bn(pretrained = True)

# すべてのパラメータで勾配計算なしに
for param in net.parameters():
    param.requires_grad = False

# 乱数初期化
torch_seed()

# 最終ノードの出力を2に変更する
# このノードのみ勾配計算をすることになる
in_features = net.classifier[6].in_features
net.classifier[6] = nn.Linear(in_features, 10)

# AdaptiveAvgPool2d関数の取り外し
net.avgpool = nn.Identity()

# GPUの利用
net = net.to(device)

# 学習率
lr = 0.001

# 損失関数定義
criterion = nn.CrossEntropyLoss()

# 最適化関数定義
# パラメータ修正の対象を最終ノードに限定
optimizer = optim.SGD(net.classifier[6].parameters(),lr=lr,momentum=0.9)

# historyファイルも同時に初期化する
history = np.zeros((0, 5))

In [ ]:
# モデルを再度定義（同じモデルを使う必要があります）
model = models.resnet18(pretrained=False, num_classes=10)

# モデルをGPU（CUDA）に転送
model.to(device)

# 保存したモデルの重みを読み込む
model.load_state_dict(torch.load(model_save_path))
print(f"Model weights loaded from {model_save_path}")

# モデルを評価モードに設定
model.eval()

# テストループ
correct = 0
total = 0
with torch.no_grad():  # 推論時は勾配計算を無効化
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the 10000 test images: {accuracy}%')

In [ ]:
evaluate_history(history)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:

        images = images.to(device)
        labels = labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 分類レポート
print(classification_report(all_labels, all_preds, target_names=classes))

# 混同行列
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# 乱数初期化
torch_seed()

# 検証データへの結果表示
show_images_labels(test_loader2, classes, net, device)

In [1]:
git remote add origin https://github.com/jun-vvn/AI_lab.git
git branch -M main
git push -u origin main

SyntaxError: invalid syntax (<ipython-input-1-4ebb1ac5fd62>, line 1)